In [1]:
"""FIRST MATCHING ALGORITHM in dev"""


#TODO
save no git hub
move the drawing part
commit and work on git hub
new circle
change variables

'FIRST MATCHING ALGORITHM in dev'

In [2]:
# modules
from system import *
from shape import *



In [3]:
# the algo module (detailed)
class MatchingAlgorithm1:
    """Store all passenger announces
    When a driver look for a passenger, compute and send the list of coherent announces (based on window time)
    When a match is made delete the announce"""
    def __init__(self,benefits_function,simu):
        self.passengerList={}
        self.benefits=benefits
        self.simulation=simu
    def addPassenger(self,agent,lastDeparture,origin,destination):
        """Add a passenger with all needed info"""
        self.passengerList[agent]={"ld":lastDeparture,"O":origin,"D":destination}
    def checkPotentialMatching(self,departureTime,origin):#information of driver
        """send the list of compatible announce (with the departureTime)"""
        out=[]
        for passenger in self.passengerList:
            p_info=self.passengerList[passenger]
            travelTime=self.simulation.network.travel_time(origin,p_info["O"])#travel time from the driver origin to the passenger origin
            if departureTime+travelTime <= p_info["ld"]:#check compatibility
                out.append(self.sendAnnounce(passenger,p_info))
        return out
    def sendAnnounce(self,agent,info):
        """set the information send to a driver when a coherent passenger is found"""
        bene=self.benefits(info["O"],info["D"])
        return (agent,{"O":info["O"],"D":info["D"],"b":bene})
    def retreivePassenger(self,agent):
        """delete an announce when accepted or obsolete"""
        if agent in self.passengerList:
            del(self.passengerList[agent])
            return True
        else:
            return False
        
        
        
        
        

        
        
#EVENT
class PublishAnnounce(Event):
    """Event when a passenger publish an announce"""
    def __init__(self, agent):
        self.time=agent.publishing_time
        self.agent=agent
    def run(self,simulation):
        agent=self.agent
        simulation.matchingAlgo.addPassenger(agent,agent.last_departure_time,agent.position,agent.destination)
        simulation.put(RetreiveAnnounce(agent,agent.last_departure_time))
        self.agent(self.time,"waiting",position=agent.position)
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
        
class RetreiveAnnounce(Event):
    """Delete a passenger's announce
        if really delete it means the passenger didn't find a proper driver
        otherwise this event is useless"""
    def __init__(self,agent,time):
        self.time=time
        self.agent=agent
    def run(self,simulation):
        if simulation.matchingAlgo.retreivePassenger(self.agent):
            self.agent(self.time,"missed")
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
        
        
class WatchAnnounce(Event):
    """A driver ask for potential passengers"""
    def __init__(self,agent,watching_time):
        self.time=watching_time
        self.agent=agent
    def __str__(self):
        return super().__str__() + " by agent " + str(self.agent.id_number)
    def run(self,simulation):
        #compute at what time the driver can leave:
        possible_departure=max(self.agent.departure_window[0],self.time)
        #look all potential matchings
        potentialMatching=simulation.matchingAlgo.checkPotentialMatching(possible_departure,self.agent.position)
        agentMatched=None
        bestRate=0
        for match in potentialMatching:
            rate=self.agent.rate(match[1])
            if rate > bestRate:#driver accept the match
                agentMatched=match[0]
        if agentMatched:#we have a match!
            simulation.matchingAlgo.retreivePassenger(agentMatched)
            self.agent(self.time,"matched",passenger=agentMatched.id_number)
            agentMatched(self.time,"matched",driver=self.agent.id_number)
            l_points=[("Od",self.agent.position),("Op",agentMatched.position),("Dp",agentMatched.destination),("Dd",self.agent.destination)]
            l_agents=[[self.agent,"Od","Dd"],[agentMatched,"Op","Dp"]]
            t=Travel(possible_departure,l_points,l_agents)
            simulation.put(t)
        else:
            next_watching=self.time+self.agent.repetition_time
            if next_watching > self.agent.departure_window[1]:#too late the driver leaves
                self.agent(self.time,"alone")
                simulation.put(Travel(possible_departure,[("Od",self.agent.position),("Dd",self.agent.destination)],[[self.agent,"Od","Dd"]]))#simplify?
            else:
                simulation.put(WatchAnnounce(self.agent,next_watching))
                self.agent(self.time,"watching",position=self.agent.position)
        
class Travel(Event):
    """Has a list of points for the trajectory
    Also has the list of travellers with their start and end points"""
    def __init__(self,time,points_list,agents_list):
        """points_list -> list of tuples (point_name,point_coordinates) in the order of the travel
        agent_list -> list of list [agent,origin_name,destination_name]
                                origin_name="" if already travelling """
        self.time=time
        self.points=points_list
        self.agents=agents_list
    def __str__(self):
        out=super().__str__()+" by agents"
        for agent in self.agents:
            out+= " " + str(agent[0].id_number)
        return out
    def run(self,simulation):
        ###first lets take a look at who arrived at destination:
        start_name,start_point=self.points[0]
        finished=[]
        for passenger in self.agents:
            if passenger[1] is "" and passenger[2] is start_name:#he was moving and arrived
                passenger[0](self.time,"arrived",point=start_point)
                finished.append(passenger)
        self.agents=[x for x in self.agents if x not in finished]
        ###then lets watch if the travel is finished
        if len(self.points) is 1:#arrived to the last point -> nothing happens
            assert len(self.agents) is 0#otherwise some people were not stopped
        else:###if not the case lets identify who started and lets built the next event
            for passenger in self.agents:
                if passenger[1] is start_name:#he begins
                    passenger[1]=""
            next_name,next_point=self.points[1]
            for passenger in self.agents:
                if passenger[1] is "":#he is moving
                    passenger[0](self.time,"moving",start=start_point,end=next_point)
            travelled_time=simulation.network.travel_time(start_point,next_point)
            del self.points[0]
            simulation.put(Travel(self.time+travelled_time,self.points,self.agents))
            
            
            
            
            
#AGENTS
class Passenger(Agent):
    """ask for a drive"""
    attributes=["publishing_time","last_departure_time","position","destination"]
    def compute(self,simulation):
        simulation.put(PublishAnnounce(self))
        
class Driver(Agent):
    """propose a drive"""
    attributes=["first_watching_time","repetition_time","departure_window","position","destination","rate"]
    def compute(self,simulation):
        simulation.put(WatchAnnounce(self,self.first_watching_time))



In [4]:
# CREATE A SIMULATION
X=(0,200)
Y=(0,300)
end=3*60
N=rectangle(X,Y,3)
T=Timer(end)
simu=Simulation(N,T)

#WITH THE ALGORITHM
passengerCostPerTime=6
fixCostOfApplication=3
def benefits(origin,destination):
    return simu.network.travel_time(origin,destination)*passengerCostPerTime+fixCostOfApplication
simu.matchingAlgo=MatchingAlgorithm1(benefits,simu)



In [5]:
# CREATE THE DEMAND
N_driver=60
N_passenger=60
def rater(*args,**kwargs):
    return 1
#GENERATORS
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Driver(first_watching_time=t,repetition_time=20,departure_window=(t+100,t+140),position=O,destination=D,rate=rater)
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,last_departure_time=t+100,position=O,destination=D)


demand=[]#just a list of agents
for i in range(N_driver):
    demand.append(SimpleDriver(simu))
for i in range(N_passenger):
    demand.append(SimplePassenger(simu))



In [6]:
#here we go
simu.add(demand)
simu.set_action(print)
simu()

Event at time 0.8201042520030266 of type WatchAnnounce by agent 24
Event at time 0.9085635135449999 of type WatchAnnounce by agent 4
Event at time 4.0614028290507225 of type WatchAnnounce by agent 10
Event at time 4.926758998479084 of type WatchAnnounce by agent 44
Event at time 5.169478520456609 of type WatchAnnounce by agent 20
Event at time 7.216898411364674 of type WatchAnnounce by agent 23
Event at time 8.126164986630135 of type PublishAnnounce by agent 88
Event at time 9.145929372180513 of type PublishAnnounce by agent 94
Event at time 9.71726616514985 of type WatchAnnounce by agent 55
Event at time 10.447945389331913 of type PublishAnnounce by agent 63
Event at time 11.259720223144203 of type WatchAnnounce by agent 11
Event at time 12.002002804040231 of type PublishAnnounce by agent 115
Event at time 12.442461837782233 of type WatchAnnounce by agent 8
Event at time 16.024067144789107 of type WatchAnnounce by agent 31
Event at time 16.777266108638383 of type PublishAnnounce by ag

In [7]:
for p in demand:
    print(p)

Agent 1 of type Driver.
begin at 372.1786747281931 to ('arrived', {'point': (181.93058003367472, 128.34024198869983)}) 
begin at 341.66349692241715 to ('moving', {'start': (145.10683448323277, 44.52737631582481), 'end': (181.93058003367472, 128.34024198869983)}) 
begin at 267.4716913541231 to ('moving', {'start': (75.81130376325846, 256.0408405293094), 'end': (145.10683448323277, 44.52737631582481)}) 
begin at 251.15064136043964 to ('moving', {'start': (61.57042195104159, 209.19441759994035), 'end': (75.81130376325846, 256.0408405293094)}) 
begin at 171.15064136043964 to ('matched', {'passenger': 84}) 
begin at 151.15064136043964 to ('watching', {'position': (61.57042195104159, 209.19441759994035)}) 

Agent 2 of type Driver.
begin at 347.3612223453056 to ('arrived', {'point': (64.2409317014104, 57.7949989422206)}) 
begin at 305.35248292073516 to ('moving', {'start': (54.20463907564994, 183.4209537553213), 'end': (64.2409317014104, 57.7949989422206)}) 
begin at 259.2905034899501 to ('mo

In [11]:
#drawing plugin



import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time


def format_time(t):#in seconds
    if t//3600>0:
        return "{}:{:02d}:{:02d}".format(int(t//3600),int((t%3600)//60),int(t%60))
    return "{:2d}:{:05.2f}".format(int(t//60),t%60)



class Drawing():
    def __init__(self,Xsize,Ysize,network,end,objects,update,**kargs):
        """
        Xsize,Ysize - > maximum of network
        network - > patch of the network's surface
        end - > finishing time
        functionPosition - > return (from t) a tuple of the list of coordinates X and Y to draw"""
        self.X=Xsize
        self.Y=Ysize
        self.surface=network
        self.finish_time=end
        self.objects=objects
        self.update_from_t=update
        self.set_options(**kargs)
    def set_options(self,margin=20,real_size=(8,8),time_interval=10,time_coef=60,repetition=True,hide_axes=False):
        """to set options: can use default values, can be furnished when __init__ is called, can be changed (even a part of it)
        margin - > around the network
        real_size - > size of the window
        time_interval - > time between each refresh (in ms)
        time_coef - > acceleration of representation (ratio)
        repetition - > (boolean) define is the animation repeat when finished
        hide_axes - > to hide axes"""
        self.margin=margin
        self.real_size=real_size
        self.time_interval=time_interval
        self.time_coef=time_coef
        self.repetition=repetition
        self.hide_axes=hide_axes
    def update_drawing(self,ob,time_text,lt):
        def function_out(frame):
            real_time=frame*self.time_interval*self.time_coef/1000
            self.update_from_t(ob,real_time)
            rl=time.time()-lt
            time_text.set_text("sl: "+format_time(real_time)+"\nsd:"+format_time(frame*self.time_interval/1000)+"\nrl: "+format_time(rl))
            return tuple(ob+[time_text])
        return function_out
    def __call__(self):
        ### set the figure
        X=self.X
        Y=self.Y
        margin=self.margin
        size=((X[0]-margin,X[1]+margin),(Y[0]-margin,Y[1]+margin))
        fig=plt.figure(figsize=(self.real_size))
        ax = fig.add_axes([0,0,1,1], frameon=False, aspect=1)
        ax.set_xlim(size[0])
        ax.set_ylim(size[1])
        if self.hide_axes :
            ax.set_xticks([]), ax.set_yticks([])
        ### draw the network
        self.surface.set_facecolor('0.9')
        ax.add_patch(self.surface)
        ### create empty draw
        object_list=self.objects(ax)
        time_text = ax.text(0, 0.9, '', transform=ax.transAxes)
        ### prepare the launching
        N_frames=int(1000*self.finish_time//self.time_coef//self.time_interval+1)#why need int?
        launching_time=time.time()
        ###launch animation
        animation = FuncAnimation(fig, self.update_drawing(object_list,time_text,launching_time), interval=self.time_interval, blit=True, repeat=self.repetition, frames=N_frames)
        plt.show()
        return animation
    
    
def Drawing_from_simulation(s,o,u):#s is a simulation object
    return Drawing(s.network.Xsize,s.network.Ysize,s.network.get_patch(),s.timer.finish_time,o,u)


In [46]:


def position(agent,t,network):
    action,to=agent.story[t]
    if action[0] is "waiting" or action[0] is "watching" :
        return action[1]["position"]
    if action[0] is "matched" :
        return position(agent,to-1,network)
    if action[0] is "moving":
        D=t-to#time since last point
        D=network.travel_distance(D)#distance made with that time
        drel=(action[1]["end"][0]-action[1]["start"][0],action[1]["end"][1]-action[1]["start"][1])#relative vector
        module=sqrt(drel[0]**2+drel[1]**2)
        dx=D*drel[0]/module#distance made in each direction
        dy=D*drel[1]/module
        return (action[1]["start"][0]+dx,action[1]["start"][1]+dy)


def positionList(agents,network):
    def out_f(t):
        out=[]
        for agent in agents:
            out.append(position(agent,t,network))
        return out
    return out_f


p=positionList(demand,simu.network)
    
def create_objects(ax):
    passenger,= ax.plot([], [],'bo', ms=4)
    driver,= ax.plot([], [],'ro', ms=6)
    return [driver,passenger]

def updateFrom(agents,network):
    def update(objects,t):
        p=[]
        d=[]
        for agent in agents:
            if isinstance(agent,Passenger):
                p.append(position(agent,t,network))
            if isinstance(agent,Driver):
                d.append(position(agent,t,network))
        if p:
            x=[x[0] for x in p if x is not None]
            y=[x[1] for x in p if x is not None]
            objects[1].set_data(x,y)
        if d:
            x=[x[0] for x in d if x is not None]
            y=[x[1] for x in d if x is not None]
            objects[0].set_data(x,y)
        return objects
    return update

update=updateFrom(demand,simu.network)

In [47]:

dr=Drawing_from_simulation(simu,create_objects,update)
dr.set_options(margin=20,real_size=(8,8),time_interval=10,time_coef=5,repetition=True,hide_axes=True)

#delete or change the use of X,Y
#propose several color, for several agent
#make it work

In [48]:
dr()